In [3]:
%serialconnect to --port COM13

Connecting to --port=COM13 --baud=115200 
Ready.


In [4]:
import os 
print(os.listdir())

['boot.py', 'ap.txt', 'main.py', 'webtool.html', 'webrepl_cfg.py', 'blink.py', 'read_dht.py', 'read_adc.py', 'blk_light.py', 'adc_light.py', 'readDHT.py']


In [5]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x419be90, open=True>(port='COM13', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [ ]:
%esptool --port COM7 erase

Executing:
  esptool --port COM7 erase_flash

esptool.py v2.6
Serial port COM7
Connecting........
[Press the PRG button now if required]


Traceback (most recent call last):
  File "D:\CSU\1072\WPy-3710-jumbo\WPy-0419-light\WPy-0419-light\python-3.7.1\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\CSU\1072\WPy-3710-jumbo\WPy-0419-light\WPy-0419-light\python-3.7.1\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\CSU\1072\WPy-3710-jumbo\WPy-0419-light\WPy-0419-light\python-3.7.1\Scripts\esptool.exe\__main__.py", line 9, in <module>
  File "D:\CSU\1072\WPy-3710-jumbo\WPy-0419-light\WPy-0419-light\python-3.7.1\lib\site-packages\esptool.py", line 2952, in _main
    main()
  File "D:\CSU\1072\WPy-3710-jumbo\WPy-0419-light\WPy-0419-light\python-3.7.1\lib\site-packages\esptool.py", line 2646, in main
    esp = ESPLoader.detect_chip(each_port, initial_baud, args.before, args.trace)
  File "D:\CSU\1072\WPy-3710-jumbo\WPy-0419-light\WPy-0419-light\python-3.7.1\lib\site-packages\esptool.py", line 259, in detect_chip
    detect_port.connect(connect_mode)
  File "D:\CSU\1

In [8]:
# esptool --port com5 --baud 460800 write_flash --flash_size=detect -fm dio 0 esp8266-20190125-v1.10.bin

usage: %esptool [--port PORT] {erase,esp32,esp8266} [binfile]

positional arguments:
  {erase,esp32,esp8266}
  binfile

optional arguments:
  --port PORT
Please download the bin file from https://micropython.org/download/#

In [33]:
%serialconnect to --port COM5 

Connecting to --port=COM5 --baud=115200 
Ready.


In [34]:
import os
print(os.listdir())

['boot.py']


In [35]:
%sendtofile ap.txt
CHT-10-5
0953313123

Sent 2 lines (19 bytes) to ap.txt.


In [36]:
%sendtofile main.txt
# -*- coding: utf-8 -*-

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = 'pypypypy'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())

#-- 當AP，並指定
uid = machine.unique_id()
#ap_if.ifconfig()
# ('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')
# (ip, mask, gateway, dns)
my_sn = '%02X-%02X-%02X-%02X' %(uid[0], uid[1], uid[2], uid[3])

#- Change name/password/ip of ESP8266's AP:
ap_if = network.WLAN(network.AP_IF)
#ap_if.ifconfig([my_ip, my_mask, my_gw, my_dns])

my_ssid = 'upy_%s_%s' %(my_sn, sta_ip)
#ap_if.config(essid = my_ssid)#改ssid，馬上生效
ap_if.config(essid=my_ssid, authmode=network.AUTH_WPA_WPA2_PSK, password="12345678")
#DHCP 功能micropython預設就有，不用設定
#AP的IP，每次重開都會回到預設值，因此需要開機時就設定
#一般是配給AP ip的下一號ip


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04


# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(0, machine.Pin.OUT)


# th_sensor
th_sensor = dht.DHT11(Pin(2))


#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))

    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    th_sensor_read = request.find('GET /?th_sensor=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)
   

    my_t = None
    my_h = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()    
        my_t = th_sensor.temperature()  
        my_h = th_sensor.humidity()
        print('T=', th_sensor.temperature())
        print('H=', th_sensor.humidity())

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?', 'T={}{}C'.format(my_t,'\u00b0'))
        else:
            html = html.replace('T=?', '')
        if my_h:
            html = html.replace('H=?', 'H={}{}'.format(my_h,'%'))
        else:
            html = html.replace('H=?', '')
        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()

Sent 138 lines (3220 bytes) to main.txt.


In [37]:
%sendtofile webtool.html
<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D4):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D1):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?unit
</form>

<hr>
GPIO Info. :<br>
<pre>
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM
</pre>

</html>

Sent 39 lines (789 bytes) to webtool.html.


In [38]:
%sendtofile webrepl_cfg.py
PASS = '1234'

Sent 1 lines (13 bytes) to webrepl_cfg.py.


In [39]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft rebooOSError: [Errno 2] ENOENT\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'